# Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm import tqdm

import time
import torch
import gc

pd.set_option('display.max_columns', None)

## Daten laden

In [ ]:
IS_TRAINING = True
if IS_TRAINING:
    FILE = '../../data/processed/train_series_split_normalized.parquet'
else:
    FILE = '../../data/processed/validation_series_split_normalized.parquet'
series = pd.read_parquet(FILE, columns=['num_series_id'])

## Features

In [ ]:
SAMPLE_FREQ = 12 # 1 min
SAMPLE_LEN = 120 # 2h

In [ ]:
LAGS_FUTURE = [f"t_lag_{i}" for i in range(-1, -SAMPLE_LEN, -1)]
LAGS_PAST = reversed([f"t_lag_{i}" for i in range(1, SAMPLE_LEN)])
FEATURES = [*LAGS_PAST, 't_0', *LAGS_FUTURE]

In [ ]:
len(FEATURES)

## Neue Features erstellen

In [ ]:
folder = "train/" if IS_TRAINING else "validation/"
file_path_prefix = "../../data/processed/transformer-downsample/" + folder

def save_chunk(num_series_id, chunk):
    series_length, series_columns = chunk[FEATURES].values.shape
    X = torch.from_numpy(np.array(np.ravel(chunk[FEATURES].values).tolist())
                           .reshape(series_length, series_columns, 2)).to(torch.float32)
    torch.save(X, file_path_prefix + str(num_series_id) + ".pt")

In [ ]:
def downsample(df):
    df['anglez'] = df['anglez'].rolling(SAMPLE_FREQ, center=True).agg('mean').bfill().ffill().values
    df['enmo'] = df['enmo'].rolling(SAMPLE_FREQ, center=True).agg('mean').bfill().ffill().values
    
    return df.iloc[::SAMPLE_FREQ, :].reset_index(drop=True)

In [ ]:
def make_features_chunk(num_series_id):
    df = pd.read_parquet(FILE, filters=[('num_series_id','=',num_series_id)])
    df = downsample(df)
    
    df['t_0'] = df[['anglez', 'enmo']].values.tolist()

    columns_past = []
    for i in range(1, SAMPLE_LEN):
        df[f'anglez_lag_{i}'] = df["anglez"].shift(i).bfill()
        df[f'enmo_lag_{i}'] = df["enmo"].shift(i).bfill()
        df[f't_lag_{i}'] = df[[f'anglez_lag_{i}', f'enmo_lag_{i}']].values.tolist()
        columns_past.extend([f'anglez_lag_{i}', f'enmo_lag_{i}'])
    df = df.drop(columns=columns_past)


    columns_future = []
    for i in range(-1, -1 * SAMPLE_LEN, -1):
        df[f'anglez_lag_{i}'] = df["anglez"].shift(i).ffill()
        df[f'enmo_lag_{i}'] = df["enmo"].shift(i).ffill()
        df[f't_lag_{i}'] = df[[f'anglez_lag_{i}', f'enmo_lag_{i}']].values.tolist()
        columns_future.extend([f'anglez_lag_{i}', f'enmo_lag_{i}'])
    df = df.drop(columns=columns_future)

    
    return df.reset_index(drop=True)

In [ ]:
def make_features(series):
    overview_data = []

    for num_series_id in tqdm(series.num_series_id.unique()):
        chunk = make_features_chunk(num_series_id)
        save_chunk(num_series_id, chunk)
        overview_data.append(
            chunk[['num_series_id', 'step', 'awake', 'critical_event_point']].reset_index().rename(columns={'index':'series_index'}).copy()[['num_series_id', 'step', 'awake', 'critical_event_point', 'series_index']]
        )
    
        del chunk
        gc.collect()
   
    return pd.concat(overview_data).reset_index(drop=True)   

In [ ]:
start_time = time.time()
overview = make_features(series)
print(f'Feature Engineering took {time.time() - start_time} seconds')

## Overview speichern

In [ ]:
overview